Funtion in python are a first class objects. Now what is a first class object?

A program entity is a first class object if it can be:
- Created at runtime
- Assigned to a variable or element in a data structure
- Passed as argument to a function 
- returned as argument to a function

In [1]:
# to demonstrate this
def factorial(n):
    """returns n!"""
    return 1 if n < 2 else n*factorial(n-1)

In [2]:
factorial(42)

1405006117752879898543142606244511569936384000000000

In [3]:
factorial.__doc__

'returns n!'

In [4]:
type(factorial)

function

In [5]:
# demonstrace the other factors that make functions a first-class obj
fact = factorial
fact

<function __main__.factorial(n)>

In [6]:
fact(5)

120

In [7]:
map(factorial, range(11))

In [8]:
list(map(factorial, range(5)))

[1, 1, 2, 6, 24]

## Higher-Order Functions

Having first-class functions enables programming in a functional style. One of the hall-marks of functional programming is the use of Higher-order functions

A function that takes another function as argument or returns a function is called a higher order function. This can be seen in the `sorted` function which has a keyword argumnet `key` which takes a function.

In [9]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [10]:
# any one-argument function can be used as the key
def reverse(word):
    return word[::-1]

reverse('testing')

'gnitset'

In [11]:
sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

`map`, `filter` and `reduce` are the best-know higher order functions. But there are better alternatives to these and we will see them below.

`listcomps` and `genexps` replace the functionalities of `map` and `filter` and in most cases they are more readable.

In [12]:
list(map(fact, range(6)))

[1, 1, 2, 6, 24, 120]

In [13]:
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [14]:
list(map(factorial, filter(lambda n: n%2, range(6))))

[1, 6, 120]

In [15]:
[factorial(n) for n in range(6) if n % 2]

[1, 6, 120]

In python3 `map` and `filter` return generators so their direct substituion is `genexps`.

In [17]:
# the reduce fun is replaced by built-in sum fun which is more fast and readable
from functools import reduce
from operator import add
reduce(add, range(100))

4950

In [18]:
sum(range(100))

4950

## Anonymous Functions

using the `lambda` keyword, you can create a anonymous function within a python expression. the `lambda` function cannot be used to create complex functions with `while` `try` etc. but its best use is in the context of arguments lists.

for example lets rewrite the reverse function we saw earlier using lambda.

In [1]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## User-defined Callable types

not only are Python functions real object, arbitary Python objects may also be made to behave like functions by implementing a `__call__` instance method.

In [6]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [13]:
bingo = BingoCage(range(3))
bingo.pick()

2

In [14]:
bingo()

0

## Function Introspection

function, as we mentioned earlier are first-class objects. Now lets look at the factors that make its so.
```
>> dir(factorial)
```
**The `__dict__` attribute**: Like instances of plain user-defined class, a function uses `__dict__` attribute to store user attributes assigned to it.

Looking closer at attributes that are specific to functions. To see the detailed explaination of each attribute ref Table-5.1 pg 148.

In [16]:
class C: pass
obj = C()

def func(): pass

sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

In [14]:
# python has an extremely flexible parameter handling mechanism
def tag(name, *content, cls=None, **attrs):
    """Generates one or more HTML tags"""
    if cls is not None:
        attrs['class'] = cls
        
    if attrs:
        attr_str = ''.join('%s="%s"' % (attr, value)
                          for attr, value
                          in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                         (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

In [18]:
tag('br')

'<br />'

In [19]:
tag('p', 'hello')

'<p>hello</p>'

In [20]:
tag('p', 'hello', 'world')

'<p>hello</p>\n<p>world</p>'

In [21]:
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
'src': 'sunset.jpg', 'cls': 'framed'}

In [22]:
tag(**my_tag)

'<imgclass="framed"src="sunset.jpg"title="Sunset Boulevard" />'

## Retrieving Information about Parameters

An interesting application of function introspection can be seen in Bobo HTTP micro-framework.

In [3]:
import bobo

@bobo.query('/')
def hello(person):
    return 'Hello %s!' % person

What's special about this is that the decorator `@bobo.query()` interospects the hello function and understands that it expects one argument. It then parses the response and return the person to the function. All this without the user having to work with the response.

This is achieved using various attributes like `__defaults__` and `__kwdefaults__`, `__code__` etc. Lets see another example to see better.

In [4]:
def clip(text, max_len=80):
    """
    Return text clipped at the last space before or afte max_len.
    """
    
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else: 
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [5]:
clip.__defaults__

(80,)

In [6]:
clip.__code__

<code object clip at 0x7f02b4437ed0, file "<ipython-input-4-c90bf6018cd7>", line 1>

In [7]:
clip.__code__.co_varnames

('text', 'max_len', 'end', 'space_before', 'space_after')

In [8]:
clip.__code__.co_argcount

2

Now this is kind off a messy way to learn about the functions but fortunately there is the `inspect` module.

In [10]:
from inspect import signature
sig = signature(clip)
sig

<Signature (text, max_len=80)>

In [11]:
str(sig)

'(text, max_len=80)'

In [13]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


as you can see this is a much better way to parse the argument info. signature also has a `annotation` attribute to give info about annotations.

`inspect.Signature` object also has a bind function that takes any number of arguments and binds them to the parameters.

In [16]:
sig = signature(tag)
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
          'src': 'sunset.jpg', 'cls': 'framed'}
bound_args = sig.bind(**my_tag)
bound_args

<BoundArguments (name='img', cls='framed', attrs={'title': 'Sunset Boulevard', 'src': 'sunset.jpg'})>

## Function Annotations

You have seen above how the Python data model with the help of inspect exposes the same machinery the interpreter uses to bind arguments to formal parameters in function call. Function annotation enhances the possible uses of this.

In [17]:
# modified with annotations

def clip(text:str, max_len:'int>0' = 0) -> str:
    """
    Return text clipped at the last space before or afte max_len.
    """
    
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else: 
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [18]:
# annotations are just additional metadata to be used by other programs
# its not used by the python interpreter

sig = signature(clip)
sig.return_annotation

str

In [19]:
for param in sig.parameters.values():
    note = repr(param.annotation).ljust(13)
    print(note, ':', param.name, '=', param.default)

<class 'str'> : text = <class 'inspect._empty'>
<class 'int'> : max_len = 0


## Packages for Functional Programming

Althou python is not a functional language, a functional coding style can be brought in by the `operator` & `functools` libs.

### `operator` module

Often in functional programming it is useful to have arithmetic operators as functions. Lets look at an example of a factorial function.

In [20]:
from functools import reduce

def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

fact(5)

120

In [21]:
# operator has these functions for you!
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

fact(5)

120

Another group of one-trick lambdas that operator replaces are functions to pick items from sequences or read attributes from objects: `itemgetter` and `attrgetter` actually build custom functions to do that.

In [1]:
metro_data = [
('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [23]:
# you can also pass multiple indexes
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


`itemgetter` supports seqences and mappings (any object with a `__getitem__`

Simalar to that is `attrgetter`, which creates a functions to extract ojbect attributes by name. 

In [4]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))

In [6]:
metro_areas[0].coord.lat

35.689722

In [9]:
from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [11]:
# all the function in operator
import operator
[name for name in dir(operator) if not name.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [12]:
from operator import methodcaller
s = 'The time has home'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS HOME'

In [14]:
hiphenate = methodcaller('replace', ' ', '-')
hiphenate(s)

'The-time-has-home'

## functools.partial

The `functools` module brings together a handful of higher-order functions. The best know function is probably `reduce`. The other most popular functions are `partial` and `partialmethod`

In [15]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
triple(7)

21

In [16]:
[triple(n) for n in range(10)]

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]

`partial` takes a callable function name and any number of positional or keyword arguments and generates a new function with the args in place. This is effective in situations where you want to pass a callback to an API that expects fewer args that that is there in the function. 

`partialmethod` does the same thing but for methods.